In [1]:
import functools

import jax
import numpy as np
import scanpy as sc

from ot_pert.metrics import compute_mean_metrics, compute_metrics

In [2]:
adata = sc.read("/lustre/groups/ml01/workspace/ot_perturbation/models/cpa/combosciplex/adata_with_predictions_30.h5ad")

/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/anndata/_core/anndata.py:1906: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [3]:
adata_train_orig = sc.read("/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_30.h5ad")

/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [4]:
adata_train_orig.uns.keys()

dict_keys(['Drug1_colors', 'Drug2_colors', 'Well_colors', 'condition_colors', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'pathway1_colors', 'pathway2_colors', 'pathway_colors', 'pca', 'rank_genes_groups', 'rank_genes_groups_cov_all', 'split_colors', 'umap'])

In [5]:
preds = adata.obsm["CPA_pred"]

In [6]:
def reconstruct_data(embedding: np.ndarray, projection_matrix: np.ndarray, mean_to_add: np.ndarray) -> np.ndarray:
    return np.matmul(embedding, projection_matrix.T) + mean_to_add


def project_data(data: np.ndarray, projection_matrix: np.ndarray, mean_to_subtract: np.ndarray) -> np.ndarray:
    return np.matmul(data - mean_to_subtract, projection_matrix)

In [7]:
OBSM_KEY_DATA_EMBEDDING = "X_pca"

In [8]:
adata_train = adata[adata.obs["split"] == "train"]
adata_test = adata[adata.obs["split"] == "test"]
adata_ood = adata[adata.obs["split"] == "ood"]

In [9]:
adata_ood.obsm["CPA_pred"].shape

(8896, 2000)

In [10]:
project_data_fn = functools.partial(
    project_data,
    projection_matrix=adata_train_orig.varm["PCs"],
    mean_to_subtract=adata_train_orig.varm["X_train_mean"].T,
)

In [11]:
test_data_target_encoded = {}
test_data_target_decoded = {}
test_data_target_encoded_predicted = {}
test_data_target_decoded_predicted = {}
for cond in adata_test.obs["condition"].cat.categories:
    if cond == "control":
        continue
    test_data_target_encoded[cond] = adata_test[adata_test.obs["condition"] == cond].obsm[OBSM_KEY_DATA_EMBEDDING]
    test_data_target_decoded[cond] = adata_test[adata_test.obs["condition"] == cond].X.A
    #pred_cpa = np.log1p(adata_test[adata_test.obs["condition"] == cond].obsm["CPA_pred"])
    pred_cpa = adata_test[adata_test.obs["condition"] == cond].obsm["CPA_pred"]
    test_data_target_decoded_predicted[cond] = pred_cpa
    test_data_target_encoded_predicted[cond] = project_data_fn(pred_cpa)


ood_data_target_encoded = {}
ood_data_target_decoded = {}
ood_data_target_encoded_predicted = {}
ood_data_target_decoded_predicted = {}
for cond in adata_ood.obs["condition"].cat.categories:
    if cond == "control":
        continue
    ood_data_target_encoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].obsm[OBSM_KEY_DATA_EMBEDDING]
    ood_data_target_decoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].X.A
    #pred_cpa = np.log1p(adata_ood[adata_ood.obs["condition"] == cond].obsm["CPA_pred"])
    pred_cpa = adata_ood[adata_ood.obs["condition"] == cond].obsm["CPA_pred"]
    ood_data_target_decoded_predicted[cond] = pred_cpa
    ood_data_target_encoded_predicted[cond] = project_data_fn(pred_cpa)

In [12]:
#train_deg_dict = {
#    k: v
#    for k, v in adata_train_orig.uns["rank_genes_groups_cov_all"].items()
#    if k in train_data_target_decoded_predicted.keys()
#}
test_deg_dict = {
    k: v
    for k, v in adata_train_orig.uns["rank_genes_groups_cov_all"].items()
    if k in test_data_target_decoded_predicted.keys()
}
ood_deg_dict = {
    k: v
    for k, v in adata_train_orig.uns["rank_genes_groups_cov_all"].items()
    if k in ood_data_target_decoded_predicted.keys()
}

In [13]:
def get_mask(x, y):
    return x[:, [gene in y for gene in adata_train.var_names]]


ood_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, ood_data_target_decoded_predicted, ood_deg_dict)
ood_deg_target_decoded = jax.tree_util.tree_map(get_mask, ood_data_target_decoded, ood_deg_dict)

test_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, test_data_target_decoded_predicted, test_deg_dict)
test_deg_target_decoded = jax.tree_util.tree_map(get_mask, test_data_target_decoded, test_deg_dict)

#train_deg_target_decoded_predicted = jax.tree_util.tree_map(
#    get_mask, train_data_target_decoded_predicted, train_deg_dict
#)
#train_deg_target_decoded = jax.tree_util.tree_map(get_mask, train_data_target_decoded, test_deg_dict)

In [14]:
deg_ood_metrics = jax.tree_util.tree_map(compute_metrics, ood_deg_target_decoded, ood_deg_target_decoded_predicted)
deg_mean_ood_metrics = compute_mean_metrics(deg_ood_metrics, prefix="deg_ood_")

deg_test_metrics = jax.tree_util.tree_map(compute_metrics, test_deg_target_decoded, test_deg_target_decoded_predicted)
deg_mean_test_metrics = compute_mean_metrics(deg_test_metrics, prefix="deg_test_")

#deg_train_metrics = jax.tree_util.tree_map(
#    compute_metrics, train_deg_target_decoded, train_deg_target_decoded_predicted
#)
#deg_mean_train_metrics = compute_mean_metrics(deg_train_metrics, prefix="deg_train_")

2024-06-13 14:58:32.310411: W external/xla/xla/service/gpu/nvptx_compiler.cc:718] The NVIDIA driver's CUDA version is 12.3 which is older than the ptxas CUDA version (12.4.131). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [15]:
#train_metrics_encoded = jax.tree_util.tree_map(
#    compute_metrics, train_data_target_encoded, train_data_target_encoded_predicted
#)
#mean_train_metrics_encoded = compute_mean_metrics(train_metrics_encoded, prefix="encoded_train_")

#train_metrics_decoded = jax.tree_util.tree_map(
#    compute_metrics, train_data_target_decoded, train_data_target_decoded_predicted
#)
#mean_train_metrics_decoded = compute_mean_metrics(train_metrics_decoded, prefix="decoded_train_")

In [16]:
pred_cpa.shape

(520, 2000)

In [17]:
test_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, test_data_target_encoded, test_data_target_encoded_predicted
)
mean_test_metrics_encoded = compute_mean_metrics(test_metrics_encoded, prefix="encoded_test_")

test_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics, test_data_target_decoded, test_data_target_decoded_predicted
)
mean_test_metrics_decoded = compute_mean_metrics(test_metrics_decoded, prefix="decoded_test_")

In [18]:
ood_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_encoded, ood_data_target_encoded_predicted
)
mean_ood_metrics_encoded = compute_mean_metrics(ood_metrics_encoded, prefix="encoded_ood_")

ood_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_decoded, ood_data_target_decoded_predicted
)
mean_ood_metrics_decoded = compute_mean_metrics(ood_metrics_decoded, prefix="decoded_ood_")

In [19]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.5569920292306364,
 'deg_ood_sinkhorn_div_1': 23.07846632003784,
 'deg_ood_sinkhorn_div_10': 15.093612098693848,
 'deg_ood_sinkhorn_div_100': 14.4687744140625,
 'deg_ood_e_distance': 28.805899734792337,
 'deg_ood_mmd': 0.3411550081787934}

In [20]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.7703001239260439,
 'decoded_test_sinkhorn_div_1': 99.94418100210336,
 'decoded_test_sinkhorn_div_10': 80.26427268981934,
 'decoded_test_sinkhorn_div_100': 23.921969487116886,
 'decoded_test_e_distance': 44.1831552919925,
 'decoded_test_mmd': 0.25218210098003685}

In [21]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.7630894829017507,
 'decoded_ood_sinkhorn_div_1': 101.22678680419922,
 'decoded_ood_sinkhorn_div_10': 69.72423400878907,
 'decoded_ood_sinkhorn_div_100': 25.246961975097655,
 'decoded_ood_e_distance': 49.59355342504298,
 'decoded_ood_mmd': 0.24642644841640465}

In [22]:
ood_metrics_decoded

{'Cediranib+PCI-34051': {'r_squared': 0.9804320002685298,
  'sinkhorn_div_1': 70.76988983154297,
  'sinkhorn_div_10': 38.915313720703125,
  'sinkhorn_div_100': 1.9930038452148438,
  'e_distance': 3.448343296989626,
  'mmd': 0.19790513921646835},
 'Givinostat+SRT1720': {'r_squared': 0.976609294850566,
  'sinkhorn_div_1': 73.07979583740234,
  'sinkhorn_div_10': 40.97629928588867,
  'sinkhorn_div_100': 2.4002151489257812,
  'e_distance': 4.265545712751546,
  'mmd': 0.20073019325937716},
 'Panobinostat+Crizotinib': {'r_squared': 0.44969316865197706,
  'sinkhorn_div_1': 143.3587188720703,
  'sinkhorn_div_10': 109.80213165283203,
  'sinkhorn_div_100': 61.93634033203125,
  'e_distance': 122.69616088717892,
  'mmd': 0.3114249287127677},
 'Panobinostat+PCI-34051': {'r_squared': 0.5018209013177218,
  'sinkhorn_div_1': 130.1132049560547,
  'sinkhorn_div_10': 96.287109375,
  'sinkhorn_div_100': 51.92755126953125,
  'e_distance': 102.80158470113771,
  'mmd': 0.2927979023121822},
 'SRT2104+Alvespimy

In [23]:
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/results/cpa"

In [31]:
import os
import pandas as pd
pd.DataFrame.from_dict(ood_metrics_encoded).to_csv(os.path.join(output_dir, "ood_metrics_encoded.csv"))
pd.DataFrame.from_dict(ood_metrics_decoded).to_csv(os.path.join(output_dir, "ood_metrics_decoded.csv"))
pd.DataFrame.from_dict(test_metrics_encoded).to_csv(os.path.join(output_dir, "test_metrics_encoded.csv"))
pd.DataFrame.from_dict(test_metrics_decoded).to_csv(os.path.join(output_dir, "test_metrics_decoded.csv"))
pd.DataFrame.from_dict(deg_test_metrics).to_csv(os.path.join(output_dir, "test_metrics_deg.csv"))
pd.DataFrame.from_dict(deg_ood_metrics).to_csv(os.path.join(output_dir, "ood_metrics_deg.csv"))
#pd.DataFrame.from_dict(train_metrics_encoded).to_csv(os.path.join(output_dir, "train_metrics_encoded.csv"))
#pd.DataFrame.from_dict(train_metrics_decoded).to_csv(os.path.join(output_dir, "train_metrics_decoded.csv"))